In [ ]:
!pip install gensim
!pip install fasttext
!pip install nltk

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import emoji
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import gensim
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
nltk.download('words')
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import h5py

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
ps = nltk.PorterStemmer()
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

# # Get the current working directory (the folder where your script is located)
# current_directory = os.getcwd()

# # Find all CSV files in the current directory
# csv_files = [file for file in os.listdir(current_directory) if file.endswith('.csv')]

# # Initialize an empty list to store individual DataFrames
# dataframes = []

# # Loop through each CSV file and read its data into a DataFrame
# # Add a new column 'File_Name' with the name of the CSV file
# for csv_file in csv_files:
#     data = pd.read_csv(csv_file)
#     csv_file=csv_file[:-5]
#     data['Category'] = csv_file  # Add a new column with the file name
#     dataframes.append(data)

# # Concatenate all DataFrames into a single DataFrame
# combined_data = pd.concat(dataframes, ignore_index=True)

# # Specify the name of the output combined CSV file
# output_csv_file = 'combined_data1.csv'

# # Save the combined data to the output CSV file
# combined_data.to_csv(output_csv_file, index=False)

# print(f'Combined data saved to {output_csv_file}')


In [3]:

df=pd.read_csv('combined_data1.csv')
print(df.shape)
print(df.isnull().sum())
df.dtypes
df['Category'].unique()

(1270, 8)
Title               0
Subscribers         0
Views               0
Likes               0
Comments            0
Description         0
Engagement Ratio    0
Category            0
dtype: int64


array(['Animation', 'Art', 'Beauty', 'Education', 'Fashion', 'Fitness',
       'Funny', 'Music', 'Pet', 'Sports', 'Technology', 'Toys',
       'Vehicles', 'VideoGames'], dtype=object)

In [4]:
def convert_to_int(x):
    x=str(x)
    if "K" in x:
        return float(x[:-1])*1000
    if "M" in x:
        return float(x[:-1])*1000000
    else:
        return float(x)
for col in ['Subscribers','Views','Likes']:
    df[col] = df[col].apply(convert_to_int)
df

,Title,Subscribers,Views,Likes,Comments,Description,Engagement Ratio,Category
0,SlayyPointOfficial,8100000.0,6500000.0,400300.0,10K,"This is Abhyudaya and Gautami, two school frie...",6.312308,Animation
1,HarshRajputofficial,4000000.0,4100000.0,314600.0,11.5K,just here to make you smile 😊😊😊 . harsh rajp...,7.953659,Animation
2,utkalcartoonworld,7500000.0,3100000.0,66300.0,953,Description NA,2.169452,Animation
3,OMENIndia,591500.0,2500000.0,141.0,9,Welcome to the official YouTube Channel of OME...,0.006000,Animation
4,WonderMunna,3200000.0,2500000.0,163400.0,583,Business: business@wondermunna.com,6.559320,Animation
...,...,...,...,...,...,...,...,...
1265,TradingChanakya,436500.0,20500.0,801.0,85,hi friends trading chanakya is a new try to p...,4.321951,VideoGames
1266,LexnosGaming,216500.0,20300.0,712.0,34,Daily Clash of Clans Esports & Ranking videos!...,3.674877,VideoGames
1267,CarbonFinGaming,402600.0,20000.0,681.0,27,Hi everyone! My name is CarbonFin and I’ve bee...,3.540000,VideoGames
1268,mairajgaming,117500.0,19800.0,1700.0,160,Hi Guys I'm Mairaj Ahmed Welcome To Mairaj Gam...,9.393939,VideoGames


In [5]:
def remove_emojis(text):
    return ''.join(char for char in text if char not in emoji.EMOJI_DATA)

def contains_non_english(text):
    try:
        text.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return True
    return False

def noDesc(text):
    if text=="Description NA":
        return True
    else:
        return False
    
column='Description'
if any(df[column].apply(emoji.emoji_count)):
    df[column] = df[column].apply(remove_emojis)  
    
non_english_mask = df[column].apply(contains_non_english)
non_desc_mask=df[column].apply(noDesc)
df=df[~non_desc_mask]
print(df.shape)
df = df[~non_english_mask]
print(df.shape)



(1198, 8)
(848, 8)


C:\Users\hp\AppData\Local\Temp\ipykernel_20924\2575002509.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[~non_english_mask]


In [6]:
print(df.loc[df['Title']=='OMENIndia']['Description'].values)

['Welcome to the official YouTube Channel of OMEN India.\nThe real hub for gamers across India!\nGet your gaming fix with fun live streams, skill & strategy guides along with news across the industry!\n']


In [7]:
product_description = "New gaming headphones , good voice modulations "
product_category=['Music',"Technology","VideoGames",'Animation']


def preprocessDescriptions(Description):
    stemmer = PorterStemmer()
    cleaned = []
    #STOPWORDS 
    words = nltk.word_tokenize(Description)
    words = [word for word in words if word.lower() not in stopwords.words('english')] 
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Join the stemmed words back into a string
    Description = ' '.join(words)
    
    Description = re.sub('http\S+', '', Description)  # links
    Description = re.sub('@[^\s]+', '', Description)  # usernames
    Description = "".join([char for char in Description if char not in string.punctuation])  # punctuations
    Description = Description.lower()  # lowercase letters
    Description = ' '.join([word for word in Description.split() if len(word) > 2])

    return Description


In [8]:

# Enable IDF (inverse document frequency) weighting
# Smooth IDF weights to prevent division by zero
# Apply sublinear scaling to term frequencies

In [9]:
def vetorize(product_description,product_category):
    df_influencers=pd.DataFrame(df.loc[df['Category'].isin(product_category)])
    df_influencers['Description']=df_influencers['Description'].apply(lambda x : preprocessDescriptions(x))
    
    tfidf_vectorizer = TfidfVectorizer(
        max_df=0.8,        
        min_df=0.01,          
        ngram_range=(1, 2),
        use_idf=True,       
        smooth_idf=True,    
        sublinear_tf=True  
    )

    tfidf_matrix = tfidf_vectorizer.fit_transform(df_influencers['Description'])
    product_tfidf = tfidf_vectorizer.transform([preprocessDescriptions(product_description)])
    
    cosine_similarities = cosine_similarity(product_tfidf, tfidf_matrix)

    compatibility_scores = (cosine_similarities * 100).flatten()

    df_influencers['CompatibilityPercentage'] = compatibility_scores
    
    min_subscribers = df_influencers['Subscribers'].min()
    max_subscribers = df_influencers['Subscribers'].max()

    # Scale the 'Subscriber' values to the range [1, 100]
    df_influencers['ScaledSubscriber'] = 1 + (df_influencers['Subscribers'] - min_subscribers) * 99 / (max_subscribers - min_subscribers)

    # Define the weights for each column
    compatibility_weight = 0.60
    subscriber_weight = 0.20
    engagement_weight = 0.20

    # Calculate the scaled combined score
    df_influencers['CombinedScore'] = (
        df_influencers['CompatibilityPercentage'] * compatibility_weight +
        df_influencers['ScaledSubscriber'] * subscriber_weight +
        df_influencers['Engagement Ratio'] * engagement_weight
    )

    # Optionally, you can sort the DataFrame by the combined score
    df_influencers = df_influencers.sort_values(by='CombinedScore', ascending=False)

    return df_influencers


In [10]:
# TF-IDF prediction dataframe
vetorize(product_description,product_category)

,Title,Subscribers,Views,Likes,Comments,Description,Engagement Ratio,Category,CompatibilityPercentage,ScaledSubscriber,CombinedScore
702,AlanChikinChow,30900000.0,3800000.0,148200.0,483,new drama series alan universe,3.912711,Music,24.439496,85.040469,32.454333
1221,Potato_,164100.0,39700.0,3100.0,49,stream good,7.931990,VideoGames,46.675111,1.441442,29.879753
1171,TotalGaming093,36400000.0,1500000.0,110000.0,3.4K,gamers ajay see playing games mobile games fun...,7.560000,VideoGames,6.317543,100.000000,25.302526
1253,fa2GOD,150500.0,23600.0,2200.0,4,valorant fa2 vlt discord discordggkzddgg7 inst...,9.338983,VideoGames,22.566001,1.404451,15.688288
1204,Wynnsanity,1500000.0,57800.0,3700.0,87,best place mobile gaming content like pubg mob...,6.551903,VideoGames,21.978181,5.074976,15.512284
...,...,...,...,...,...,...,...,...,...,...,...
956,TataHitachiCorporate,22000.0,26600.0,53.0,0,tata hitachi construction machinery company pr...,0.199248,Technology,0.000000,1.054942,0.250838
1001,GAILIndiaLimited,6400.0,10400.0,9.0,0,india natural gas leader energizingpossibilities,0.086538,Technology,0.000000,1.012512,0.219810
937,starcement893,6800.0,85300.0,31.0,2,official whatsapp number 7595080005,0.038687,Technology,0.000000,1.013600,0.210457
960,TargetPEAKApp,4000.0,21800.0,6.0,0,target peak helping students especially state ...,0.027523,Technology,0.000000,1.005984,0.206701


In [11]:
#Word to Vec Method
def average_word2vec(tokens, model, num_features):
    word_vectors = [model.wv[word] for word in tokens if word in model.wv.index_to_key]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(num_features, dtype=np.float32)
def wordToVec(product_description,product_category):   
    df_influencers = pd.DataFrame(df.loc[df['Category'].isin(product_category)])
    df_influencers['Description'] = df_influencers['Description'].apply(lambda x : preprocessDescriptions(x))
    product_description = preprocessDescriptions(product_description)

    df_influencers['Description'] = df_influencers['Description'].apply(lambda x : nltk.word_tokenize(x))
    product_description = nltk.word_tokenize(product_description)
    model = Word2Vec(df_influencers['Description'], vector_size=200, window=5, min_count=1, sg=0,negative=5,alpha=0.01)
    df_influencers['InfluencerVector'] = df_influencers['Description'].apply(
        lambda x: average_word2vec(x, model, num_features=50))
    product_vector = average_word2vec(product_description, model, num_features=50)
    cosine_similarities = cosine_similarity([product_vector], list(df_influencers['InfluencerVector']))[0]

    compatibility_scores = (cosine_similarities * 100).tolist()
    df_influencers['CompatibilityPercentage'] = compatibility_scores
    min_subscribers = df_influencers['Subscribers'].min()
    max_subscribers = df_influencers['Subscribers'].max()
 
    df_influencers['ScaledSubscriber'] = 1 + (df_influencers['Subscribers'] - min_subscribers) * 99 / (max_subscribers - min_subscribers)

    compatibility_weight = 0.60
    subscriber_weight = 0.20
    engagement_weight = 0.20
    df_influencers['CombinedScore'] = (
        df_influencers['CompatibilityPercentage'] * compatibility_weight +
        df_influencers['ScaledSubscriber'] * subscriber_weight +
        df_influencers['Engagement Ratio'] * engagement_weight
    )
    df_influencers = df_influencers.sort_values(by='CombinedScore', ascending=False)
    return df_influencers




In [12]:
wordToVec(product_description,product_category)

,Title,Subscribers,Views,Likes,Comments,Description,Engagement Ratio,Category,InfluencerVector,CompatibilityPercentage,ScaledSubscriber,CombinedScore
702,AlanChikinChow,30900000.0,3800000.0,148200.0,483,"[new, drama, series, alan, universe]",3.912711,Music,"[9.404253e-05, -0.0025730769, 0.00070187566, 0...",22.525171,85.040469,31.305739
1242,KaztroGAMINGYT,1400000.0,26600.0,3700.0,30,"[kaztro, gaming, upload, gaming, videos, lives...",14.022556,VideoGames,"[0.0009786173, 0.00081373355, 0.00023822165, 0...",34.925426,4.802985,24.720364
1206,RONGAMING72,4500000.0,55900.0,3700.0,122,"[hiii, guys, name, ron, like, playing, games, ...",6.837209,VideoGames,"[-0.00028199202, 0.00055687426, 0.0001047884, ...",33.241714,13.234720,23.959415
1171,TotalGaming093,36400000.0,1500000.0,110000.0,3.4K,"[gamers, ajay, see, playing, games, mobile, ga...",7.560000,VideoGames,"[-0.00042640246, -0.0002455811, -0.00041065682...",3.466734,100.000000,23.592040
1221,Potato_,164100.0,39700.0,3100.0,49,"[stream, good]",7.931990,VideoGames,"[0.0015290827, 0.0040143407, 0.00032897887, -0...",34.870667,1.441442,22.797086
...,...,...,...,...,...,...,...,...,...,...,...,...
1236,Jokesta,644400.0,30300.0,1400.0,101,"[jokesta, handheld, gamer, plays, call, duty, ...",4.953795,VideoGames,"[0.0010129698, -0.0025729763, -0.00088011526, ...",-12.412761,2.747817,-5.907334
984,krishnaik06,819500.0,13400.0,485.0,30,"[cofounder, ineuron, experience, pioneering, m...",3.843284,Technology,"[4.809658e-05, -6.77834e-05, 0.00011969127, -8...",-13.334128,3.224074,-6.587005
970,AstrologerMohnish,18600.0,17000.0,119.0,15,"[channel, dedicated, divine, science, vedic, n...",0.788235,Technology,"[0.0007991669, -0.0005749547, 0.00029578077, -...",-12.415857,1.045695,-7.082728
707,RUNUPRECORDS,2900000.0,762700.0,25100.0,826,"[record, label]",3.399240,Music,"[-0.0003106338, 0.0033271874, -0.00067426334, ...",-15.973568,8.882857,-7.127721


In [17]:
product_description = "Headphones for gamings, surround sound high bass , suitable for FPS games."
product_category=['Music',"Technology","VideoGames",'Animation']


def prediction(product_description,product_category):
    word_2_vec_df=wordToVec(product_description,product_category)
    tfidf_vec= vetorize(product_description,product_category)   
    count=5 
    word_2_vec_df=word_2_vec_df[['Title','Subscribers','Engagement Ratio','CompatibilityPercentage','CombinedScore']]
    tfidf_vec=tfidf_vec[['Title','Subscribers','Engagement Ratio','CompatibilityPercentage','CombinedScore']]
    
    return word_2_vec_df.head(count),tfidf_vec.head(count)
        

In [18]:
word_2_vec_df,tfidf_vec=prediction(product_description,product_category)

In [19]:
word_2_vec_df

,Title,Subscribers,Engagement Ratio,CompatibilityPercentage,CombinedScore
1171,TotalGaming093,36400000.0,7.560000,28.368469,38.533082
1183,GyanGaming,14900000.0,8.326154,30.829025,28.467013
1180,Panda.,12300000.0,12.201382,30.041672,27.355290
38,Panda.,12300000.0,12.201382,30.041672,27.355290
1264,Squally,248500.0,6.333333,27.533791,18.121142


In [20]:
tfidf_vec

,Title,Subscribers,Engagement Ratio,CompatibilityPercentage,CombinedScore
1171,TotalGaming093,36400000.0,7.560000,14.281901,30.081141
1180,Panda.,12300000.0,12.201382,16.664499,19.328987
38,Panda.,12300000.0,12.201382,16.664499,19.328987
1183,GyanGaming,14900000.0,8.326154,14.099546,18.429325
1268,mairajgaming,117500.0,9.393939,26.173286,17.845698
